##Task 1. Prework - Set up environment, enable APIs and create clusters

In [ ]:
# Set up environment variables for your Project ID (this is important as it is used in several of the configuration files below)
export PROJECT_ID=$(gcloud config get-value project)
export PROJECT_NUMBER=$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')
export REGION=us-east4
gcloud config set compute/region $REGION

In [ ]:
# Enable needed services in the Google Cloud project
gcloud services enable \
container.googleapis.com \
clouddeploy.googleapis.com \
artifactregistry.googleapis.com \
cloudbuild.googleapis.com

In [ ]:
# Enable permissions for both Kubernetes and Cloud Deploy
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member=serviceAccount:$(gcloud projects describe $PROJECT_ID \
--format="value(projectNumber)")-compute@developer.gserviceaccount.com \
--role="roles/clouddeploy.jobRunner"

gcloud projects add-iam-policy-binding $PROJECT_ID \
--member=serviceAccount:$(gcloud projects describe $PROJECT_ID \
--format="value(projectNumber)")-compute@developer.gserviceaccount.com \
--role="roles/container.developer"

In [ ]:
# Create an Artifact Repository
gcloud artifacts repositories create cicd-challenge \
--description="Image registry for tutorial web app" \
--repository-format=docker \
--location=$REGION

In [ ]:
# Create the Google Kubernetes Engine clusters
gcloud container clusters create cd-staging --node-locations=us-east4-a --num-nodes=1 --async
gcloud container clusters create cd-production --node-locations=us-east4-a --num-nodes=1 --async

##Task 2. Build the images and upload to the repository

In [ ]:
# Clone the repository for the lab into home directory
cd ~/
git clone https://github.com/GoogleCloudPlatform/cloud-deploy-tutorials.git
cd cloud-deploy-tutorials
git checkout c3cae80 --quiet
cd tutorials/base

In [ ]:
# Create the skaffold.yaml configuration
envsubst < clouddeploy-config/skaffold.yaml.template > web/skaffold.yaml
cat web/skaffold.yaml

In [ ]:
# Run the skaffold command to build the application and deploy the container image to the Artifact Registry repository previously created
cd web
skaffold build --interactive=false \
--default-repo $REGION-docker.pkg.dev/$DEVSHELL_PROJECT_ID/cicd-challenge \
--file-output artifacts.json
cd ..

##Task 3. Create the Delivery Pipeline

In [ ]:
# Create the delivery-pipeline resource using the delivery-pipeline.yaml file
cp clouddeploy-config/delivery-pipeline.yaml.template clouddeploy-config/delivery-pipeline.yaml
sed -i "s/targetId: staging/targetId: cd-staging/" clouddeploy-config/delivery-pipeline.yaml
sed -i "s/targetId: prod/targetId: cd-production/" clouddeploy-config/delivery-pipeline.yaml
sed -i "/targetId: test/d" clouddeploy-config/delivery-pipeline.yaml

In [ ]:
# Set the deployment region using the deploy/region configuration parameter
gcloud config set deploy/region $REGION

In [ ]:
# Apply the pipeline configuration you created above using thef gcloud beta deploy command
gcloud beta deploy apply --file=clouddeploy-config/delivery-pipeline.yaml

In [ ]:
# Verify the delivery pipeline was created
gcloud beta deploy delivery-pipelines describe web-app

Configure the deployment targets

In [ ]:
# Ensure that the clusters are ready
gcloud container clusters list --format="csv(name,status)"

In [ ]:
# Create a context for each cluster
CONTEXTS=("cd-staging" "cd-production" )
for CONTEXT in ${CONTEXTS[@]}
do
    gcloud container clusters get-credentials ${CONTEXT} --region ${REGION}
    kubectl config rename-context gke_${PROJECT_ID}_${REGION}_${CONTEXT} ${CONTEXT}
done

In [ ]:
#  Create a namespace in each cluster
for CONTEXT in ${CONTEXTS[@]}
do
    kubectl --context ${CONTEXT} apply -f kubernetes-config/web-app-namespace.yaml
done

In [ ]:
# Create the delivery pipeline targets
envsubst < clouddeploy-config/target-staging.yaml.template > clouddeploy-config/target-cd-staging.yaml
envsubst < clouddeploy-config/target-prod.yaml.template > clouddeploy-config/target-cd-production.yaml

sed -i "s/staging/cd-staging/" clouddeploy-config/target-cd-staging.yaml
sed -i "s/prod/cd-production/" clouddeploy-config/target-cd-production.yaml

In [ ]:
# Apply the target files to Cloud Deploy
for CONTEXT in ${CONTEXTS[@]}
do
    envsubst < clouddeploy-config/target-$CONTEXT.yaml.template > clouddeploy-config/target-$CONTEXT.yaml
    gcloud beta deploy apply --file clouddeploy-config/target-$CONTEXT.yaml
done

In [ ]:
# Display the details for the staging target
cat clouddeploy-config/target-cd-staging.yaml

##Task 4. Create a Release

In [ ]:
# Create a release using the gcloud beta deploy releases command and the skaffold and artifacts.json files
gcloud beta deploy releases create web-app-001 \
--delivery-pipeline web-app \
--build-artifacts web/artifacts.json \
--source web/

In [ ]:
# Verify that your application has been deployed to the staging environment (cd-staging)
gcloud beta deploy rollouts list \
--delivery-pipeline web-app \
--release web-app-001

##Task 5. Promote your application to production

In [ ]:
# Promote your application from the Staging (cd-staging) environment to the Production (cd-production) environment
gcloud beta deploy releases promote \
--delivery-pipeline web-app \
--release web-app-001 \
--quiet

In [ ]:
# Approve the deployment
gcloud beta deploy rollouts approve web-app-001-to-cd-production-0001 \
--delivery-pipeline web-app \
--release web-app-001 \
--quiet

##Task 6. Make a change to the application and redeploy it

In [ ]:
# Using the editor, open the cloud-deploy-tutorials/tutorials/base/web/leeroy-app/ directory and modify the app.go file.
# Change line 24 to say: fmt.Fprintf(w, "leeroooooy app v2!!\n")
gcloud services enable cloudbuild.googleapis.com
# Open Editor

In [ ]:
# Build the application and push to the Artifact Registry
cd web
skaffold build --interactive=false \
--default-repo $REGION-docker.pkg.dev/$DEVSHELL_PROJECT_ID/cicd-challenge \
--file-output artifacts.json
cd ..

In [ ]:
# Create a new release on your pipeline you created earlier. Name the release web-app-002
gcloud beta deploy releases create web-app-002 \
--delivery-pipeline web-app \
--build-artifacts web/artifacts.json \
--source web/

In [ ]:
# Verify the new version has been deployed to the staging environment
gcloud beta deploy rollouts list \
--delivery-pipeline web-app \
--release web-app-002

##Task 7. Rollback The Change

In [ ]:
# Use Cloud Deploy to rollback to the original version of the application - web-app-001
gcloud deploy targets rollback cd-staging \
   --delivery-pipeline=web-app \
   --quiet